# Goal

Accuracy as a function of isotope incorporation

### Variable parameters:

* atom % isotope incorporation
  * 0, 25, 50, 100
* % taxa that incorporate
  * __1, 5, 10, 25, 50__
* n-reps (stocastic: taxon abundances & which incorporate)
  * 10 

## Init

In [42]:
import os
import glob
import itertools
import nestly

In [43]:
%load_ext rpy2.ipython
%load_ext pushnote

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The pushnote extension is already loaded. To reload it, use:
  %reload_ext pushnote


In [44]:
%%R
library(ggplot2)
library(dplyr)
library(tidyr)
library(gridExtra)

### BD min/max

In [45]:
## min G+C cutoff
min_GC = 13.5
## max G+C cutoff
max_GC = 80
## max G+C shift
max_13C_shift_in_BD = 0.036


min_BD = min_GC/100.0 * 0.098 + 1.66    
max_BD = max_GC/100.0 * 0.098 + 1.66    

max_BD = max_BD + max_13C_shift_in_BD

print 'Min BD: {}'.format(min_BD)
print 'Max BD: {}'.format(max_BD)

Min BD: 1.67323
Max BD: 1.7744


# Nestly

* assuming fragments already simulated

In [46]:
# paths
workDir = '/home/nick/notebook/SIPSim/dev/bac_genome1147/'
buildDir = os.path.join(workDir, 'atomIncorp_taxaIncorp')
R_dir = '/home/nick/notebook/SIPSim/lib/R/'

fragFile = '/home/nick/notebook/SIPSim/dev/bac_genome1147/validation/ampFrags_kde.pkl'
genome_index = '/var/seq_data/ncbi_db/genome/Jan2016/bac_complete_spec-rep1_rn/genome_index.txt'

In [47]:
if not os.path.isdir(buildDir):
    os.makedirs(buildDir)
%cd $buildDir

/home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp


In [48]:
# making an experimental design file for qSIP
x = range(1,7)
y = ['control', 'treatment']

expDesignFile = os.path.join(buildDir, 'qSIP_exp_design.txt')
with open(expDesignFile, 'wb') as outFH:
    for i,z in itertools.izip(x,itertools.cycle(y)):
        line = '\t'.join([str(i),z])
        outFH.write(line + '\n')

!head $expDesignFile       

1	control
2	treatment
3	control
4	treatment
5	control
6	treatment


## Nestly params

In [49]:
# building tree structure
nest = nestly.Nest()

# varying params: test
#nest.add('percIncorp', [100])
#nest.add('percTaxa', [10])
#nest.add('rep', range(1,4))

# varying params
nest.add('percIncorp', [0, 25, 50, 100])
nest.add('percTaxa', [1, 5, 10, 25, 50])
nest.add('rep', range(1,11))

## set params
nest.add('abs', ['1e9'], create_dir=False)
nest.add('np', [10], create_dir=False)
nest.add('Monte_rep', [100000], create_dir=False)
nest.add('subsample_dist', ['lognormal'], create_dir=False)
nest.add('subsample_mean', [9.432], create_dir=False)
nest.add('subsample_scale', [0.5], create_dir=False)
nest.add('subsample_min', [10000], create_dir=False)
nest.add('subsample_max', [30000], create_dir=False)
nest.add('min_BD', [min_BD], create_dir=False)
nest.add('max_BD', [max_BD], create_dir=False)
nest.add('DBL_scaling', [0.5], create_dir=False)
nest.add('bandwidth', [0.8], create_dir=False)
nest.add('heavy_BD_min', [1.71], create_dir=False)
nest.add('heavy_BD_max', [1.75], create_dir=False)
nest.add('topTaxaToPlot', [100], create_dir=False)
nest.add('padj', [0.1], create_dir=False)
nest.add('log2', [0.25], create_dir=False)

### input/output files
nest.add('buildDir', [buildDir], create_dir=False)
nest.add('R_dir', [R_dir], create_dir=False)
nest.add('genome_index', [genome_index], create_dir=False)
nest.add('fragFile', [fragFile], create_dir=False)
nest.add('exp_design', [expDesignFile], create_dir=False)


# building directory tree
nest.build(buildDir)

# bash file to run
bashFile = os.path.join(buildDir, 'SIPSimRun.sh')

## Experimental design

In [72]:
bashFileTmp = os.path.splitext(bashFile)[0] + '_expDesign.sh'
bashFileTmp

'/home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/SIPSimRun_expDesign.sh'

In [73]:
%%writefile $bashFileTmp
#!/bin/bash

echo '#-- Experimental design --#'

echo '# Making an isotope incorporation config file'
echo '## 3 replicate gradients for control & treatment'
SIPSim incorpConfigExample \
  --percIncorpUnif {percIncorp} \
  --n_reps 3 \
  > incorp.config

echo '# Selecting incorporator taxa'
echo '## This is to make the gradient replicates consistent (qSIP finds mean among replicates)'
SIPSim KDE_selectTaxa \
    -p {percTaxa} \
    {fragFile} \
    > incorporators.txt

echo '# Creating a community file (3 replicate control, 3 replicate treatment)'
SIPSim communities \
    --config incorp.config \
    {genome_index} \
    > comm.txt    

echo '# simulating gradient fractions'
SIPSim gradient_fractions \
    --BD_min {min_BD} \
    --BD_max {max_BD} \
    comm.txt \
    > fracs.txt        

Writing /home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/SIPSimRun_expDesign.sh


In [51]:
!chmod 777 $bashFileTmp
!cd $workDir; \
    nestrun --template-file $bashFileTmp -d $buildDir --log-file exp_design.log -j 10

2016-03-28 06:59:24,041 * INFO * Template: ./SIPSimRun.sh
2016-03-28 06:59:24,043 * INFO * [145094] Started ./SIPSimRun.sh in atomIncorp_taxaIncorp/50/50/7
2016-03-28 06:59:24,044 * INFO * [145095] Started ./SIPSimRun.sh in atomIncorp_taxaIncorp/50/50/9
2016-03-28 06:59:24,045 * INFO * [145097] Started ./SIPSimRun.sh in atomIncorp_taxaIncorp/50/50/8
2016-03-28 06:59:24,047 * INFO * [145099] Started ./SIPSimRun.sh in atomIncorp_taxaIncorp/50/50/3
2016-03-28 06:59:24,048 * INFO * [145101] Started ./SIPSimRun.sh in atomIncorp_taxaIncorp/50/50/10
2016-03-28 06:59:24,050 * INFO * [145103] Started ./SIPSimRun.sh in atomIncorp_taxaIncorp/50/50/2
2016-03-28 06:59:24,052 * INFO * [145105] Started ./SIPSimRun.sh in atomIncorp_taxaIncorp/50/50/1
2016-03-28 06:59:24,054 * INFO * [145107] Started ./SIPSimRun.sh in atomIncorp_taxaIncorp/50/50/4
2016-03-28 06:59:24,056 * INFO * [145109] Started ./SIPSimRun.sh in atomIncorp_taxaIncorp/50/50/5
2016-03-28 06:59:24,058 * INFO * [145111] Started ./SIPSimR

In [52]:
%pushnote exp_design complete: $buildDir

## SIPSim pipeline

In [75]:
bashFileTmp = os.path.splitext(bashFile)[0] + '_SIPSim-pipeline.sh'
bashFileTmp

'/home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/SIPSimRun_SIPSim-pipeline.sh'

In [76]:
%%writefile $bashFileTmp
#!/bin/bash

echo '#-- SIPSim pipeline --#'    
    
echo '# Adding diffusion'    
SIPSim diffusion \
    -n {Monte_rep} \
    --bw {bandwidth} \
    --np {np} \
    {fragFile} \
    > ampFrags_KDE_dif.pkl    

echo '# Adding DBL contamination; abundance-weighted smearing'
SIPSim DBL \
    -n {Monte_rep} \
    --comm comm.txt \
    --commx {DBL_scaling} \
    --np {np} \
    ampFrags_KDE_dif.pkl \
    > ampFrags_KDE_dif_DBL.pkl 

echo '# Adding isotope incorporation to BD distribution'
SIPSim isotope_incorp \
    -n {Monte_rep} \
    --comm comm.txt \
    --taxa incorporators.txt \
    --np {np} \
    ampFrags_KDE_dif_DBL.pkl \
    incorp.config \
    > ampFrags_KDE_dif_DBL_inc.pkl

echo '# Simulating an OTU table'
SIPSim OTU_table \
    --abs {abs} \
    --np {np} \
    ampFrags_KDE_dif_DBL_inc.pkl \
    comm.txt \
    fracs.txt \
    > OTU_abs{abs}.txt
    
echo '# Simulating PCR'
SIPSim OTU_PCR \
    OTU_abs{abs}.txt \
    > OTU_abs{abs}_PCR.txt    
    
echo '# Subsampling from the OTU table (simulating sequencing of the DNA pool)'
SIPSim OTU_subsample \
    --dist {subsample_dist} \
    --dist_params mean:{subsample_mean},sigma:{subsample_scale} \
    --min_size {subsample_min} \
    --max_size {subsample_max} \
    OTU_abs{abs}_PCR.txt \
    > OTU_abs{abs}_PCR_sub.txt
        
echo '# Making a wide-formatted table'
SIPSim OTU_wideLong -w \
    OTU_abs{abs}_PCR_sub.txt \
    > OTU_abs{abs}_PCR_sub_w.txt
    
echo '# Making metadata (phyloseq: sample_data)'
SIPSim OTU_sampleData \
    OTU_abs{abs}_PCR_sub.txt \
    > OTU_abs{abs}_PCR_sub_meta.txt
       

#-- removing large intermediate files --#
rm -f ampFrags_KDE_dif.pkl
rm -f ampFrags_KDE_dif_DBL.pkl
rm -f ampFrags_KDE_dif_DBL_inc.pkl    

Writing /home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/SIPSimRun_SIPSim-pipeline.sh


In [ ]:
!chmod 777 $bashFileTmp
!cd $workDir; \
    nestrun --template-file $bashFileTmp -d $buildDir --log-file SIPSim_pipeline.log -j 2

2016-03-28 07:01:04,305 * INFO * Template: ./SIPSimRun.sh
2016-03-28 07:01:04,307 * INFO * [167960] Started ./SIPSimRun.sh in atomIncorp_taxaIncorp/50/50/7
2016-03-28 07:01:04,308 * INFO * [167961] Started ./SIPSimRun.sh in atomIncorp_taxaIncorp/50/50/9
2016-03-28 07:32:58,553 * INFO * [167961] atomIncorp_taxaIncorp/50/50/9 Finished with 0
2016-03-28 07:32:58,631 * INFO * [168813] Started ./SIPSimRun.sh in atomIncorp_taxaIncorp/50/50/8
2016-03-28 07:38:10,926 * INFO * [167960] atomIncorp_taxaIncorp/50/50/7 Finished with 0
2016-03-28 07:38:10,952 * INFO * [169053] Started ./SIPSimRun.sh in atomIncorp_taxaIncorp/50/50/3
2016-03-28 08:06:02,932 * INFO * [168813] atomIncorp_taxaIncorp/50/50/8 Finished with 0
2016-03-28 08:06:02,954 * INFO * [170001] Started ./SIPSimRun.sh in atomIncorp_taxaIncorp/50/50/10
2016-03-28 08:08:42,327 * INFO * [169053] atomIncorp_taxaIncorp/50/50/3 Finished with 0
2016-03-28 08:08:42,345 * INFO * [170196] Started ./SIPSimRun.sh in atomIncorp_taxaIncorp/50/50/2
2

In [ ]:
%pushnote SIPSim pipeline complete: $buildDir

## Summary of simulated data

In [78]:
bashFileTmp = os.path.splitext(bashFile)[0] + '_SIPSim-summary.sh'
bashFileTmp

'/home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/SIPSimRun_SIPSim-summary.sh'

In [79]:
%%writefile $bashFileTmp
#!/bin/bash
   
# plotting 'raw' taxon abundances
SIPSimR OTU_taxonAbund \
    OTU_abs{abs}.txt \
    -r {topTaxaToPlot} \
    -o OTU_abs{abs}

# plotting 'sequenced' taxon abundances
SIPSimR OTU_taxonAbund \
    OTU_abs{abs}_PCR_sub.txt \
    -r {topTaxaToPlot} \
    -o OTU_abs{abs}_PCR_sub

Writing /home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/SIPSimRun_SIPSim-summary.sh


In [ ]:
!chmod 777 $bashFileTmp
!cd $workDir; \
    nestrun --template-file $bashFileTmp -d $buildDir --log-file SIPSim_summary.log -j 10

## HR-SIP: DESeq2

In [81]:
bashFileTmp = os.path.splitext(bashFile)[0] + '_HRSIP.sh'
bashFileTmp

'/home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/SIPSimRun_HRSIP.sh'

In [82]:
%%writefile $bashFileTmp
#!/bin/bash

# phyloseq
## making phyloseq object from OTU table
SIPSimR phyloseq_make \
    OTU_abs{abs}_PCR_sub_w.txt \
    -s OTU_abs{abs}_PCR_sub_meta.txt \
    > OTU_abs{abs}_PCR_sub.physeq

## filtering phyloseq object to just 'heavy' fractions
SIPSimR phyloseq_edit \
    OTU_abs{abs}_PCR_sub.physeq \
    --BD_min {heavy_BD_min} \
    --BD_max {heavy_BD_max} \
    > OTU_abs{abs}_PCR_sub_filt.physeq

## making ordination
SIPSimR phyloseq_ordination \
    OTU_abs{abs}_PCR_sub_filt.physeq \
    OTU_abs{abs}_PCR_sub_filt_bray-NMDS.pdf

# DESeq2
SIPSimR phyloseq_DESeq2 \
    --log2 {log2} \
    --hypo greater \
    --cont 1,3,5 \
    --treat 2,4,6 \
    OTU_abs{abs}_PCR_sub_filt.physeq \
    > OTU_abs{abs}_PCR_sub_filt_DESeq2

Writing /home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/SIPSimRun_HRSIP.sh


In [ ]:
!chmod 777 $bashFileTmp
!cd $workDir; \
    nestrun --template-file $bashFileTmp -d $buildDir --log-file HR-SIP.log -j 10

In [ ]:
%pushnote HR-SIP complete: $buildDir

## qSIP

In [84]:
bashFileTmp = os.path.splitext(bashFile)[0] + '_qSIP.sh'
bashFileTmp

'/home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/SIPSimRun_qSIP.sh'

In [85]:
%%writefile $bashFileTmp
#!/bin/bash


# qSIP
SIPSim qSIP \
    OTU_abs{abs}.txt \
    OTU_abs{abs}_PCR_sub.txt \
    > OTU_abs{abs}_PCR_sub_qSIP.txt
        

# qSIP: atom excess
SIPSim qSIP_atomExcess \
    --np {np} \
    OTU_abs{abs}_PCR_sub_qSIP.txt \
    {exp_design} \
    > OTU_abs{abs}_PCR_sub_qSIP_atom.txt  

Writing /home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/SIPSimRun_qSIP.sh


In [ ]:
!chmod 777 $bashFileTmp
!cd $workDir; \
    nestrun --template-file $bashFileTmp -d $buildDir --log-file qSIP.log -j 2

In [ ]:
%pushnote qSIP complete: $buildDir

## Making confusion matrices

In [87]:
bashFileTmp = os.path.splitext(bashFile)[0] + '_cMtx.sh'
bashFileTmp

'/home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/SIPSimRun_cMtx.sh'

In [88]:
%%writefile $bashFileTmp
#!/bin/bash

SIPSimR DESeq2_confuseMtx \
    --libs 2,4,6 \
    --padj {padj} \
    BD-shift_stats.txt \
    OTU_abs{abs}_PCR_sub_filt_DESeq2
    
SIPSimR qSIP_confuseMtx \
    --libs 2,4,6 \
    BD-shift_stats.txt \
    OTU_abs{abs}_PCR_sub_qSIP_atom.txt
     
SIPSimR heavy_confuseMtx \
    --libs 2,4,6 \
    BD-shift_stats.txt \
    OTU_abs{abs}_PCR_sub.txt

Writing /home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/SIPSimRun_cMtx.sh


In [ ]:
!chmod 777 $bashFileTmp
!cd $workDir; \
    nestrun --template-file $bashFileTmp -d $buildDir --log-file cMtx.log -j 10

### Aggregating the confusion matrix data

In [ ]:
def agg_cMtx(prefix):
    # all data
    !nestagg delim \
        -d $buildDir \
        -k percIncorp,percTaxa,rep \
        -o $prefix-cMtx_data.txt \
        --tab \
        $prefix-cMtx_data.txt

    # overall
    !nestagg delim \
        -d $buildDir \
        -k percIncorp,percTaxa,rep \
        -o $prefix-cMtx_overall.txt \
        --tab \
        $prefix-cMtx_overall.txt

    # by class
    !nestagg delim \
        -d $buildDir \
        -k percIncorp,percTaxa,rep \
        -o $prefix-cMtx_byClass.txt \
        --tab \
        $prefix-cMtx_byClass.txt
        
agg_cMtx('DESeq2') 
agg_cMtx('qSIP') 
agg_cMtx('heavy') 

In [ ]:
%pushnote atomIncorp_taxaIncorp complete!

***
# --End of simulation--#
***

# Plotting results

In [ ]:
F = os.path.join(buildDir, '*-cMtx_byClass.txt')
files = glob.glob(F)
files

In [ ]:
%%R -i files

df_byClass = list()
for (f in files){
    ff = strsplit(f, '/') %>% unlist
    fff = ff[length(ff)]
    df_byClass[[fff]] = read.delim(f, sep='\t')
}

df_byClass = do.call(rbind, df_byClass)
df_byClass$file = gsub('\\.[0-9]+$', '', rownames(df_byClass))
df_byClass$method = gsub('-.+', '', df_byClass$file)
rownames(df_byClass) = 1:nrow(df_byClass)

df_byClass %>% head(n=3)

In [ ]:
%%R -w 800 -h 550
# summarize by SIPSim rep & library rep
df_byClass.s = df_byClass %>%
    group_by(method, percIncorp, percTaxa, variables) %>%
    summarize(mean_value = mean(values),
              sd_value = sd(values))

# plotting
ggplot(df_byClass.s, aes(variables, mean_value, color=method,
                         ymin=mean_value-sd_value,
                         ymax=mean_value+sd_value)) +
    geom_pointrange(alpha=0.8) +
    labs(y='Value') +
    facet_grid(percIncorp ~ percTaxa) +
    theme_bw() +
    theme(
        text = element_text(size=16),
        axis.title.x = element_blank(),
        axis.text.x = element_text(angle=45, hjust=1)
    )

# --re-run--

In [93]:
import glob

## Function for detecting missing/empty files

In [154]:
tmp = '/home/test/'
os.path.split(tmp)

('/home/test', '')

In [164]:
def find_missing_files(filepath):
    # directories
    dirpath = os.path.split(filepath)[0]
    chk = os.path.join(buildDir, dirpath) + '/'
    D = set([os.path.split(x)[0] for x in glob.glob(chk)])
    
    # files
    chk = os.path.join(buildDir, filepath)
    F = glob.glob(chk)
    
    # check for missing files
    Fd = set([os.path.split(f)[0] for f in F])
    missing = D - Fd    
    print 'Union length: {}'.format(len(D | Fd))
    print 'Number of missing: {}'.format(len(missing))

    # check for empty files
    empties = [os.path.split(f)[0] for f in F if os.path.getsize(f) < 1000]
    print 'Number of empties: {}'.format(len(empties))
    
    # ret
    return {'missing' : list(missing), 'empty' : empties}
    
find_missing_files('*/*/*/OTU_abs1e9.txt')    

Union length: 200
Number of missing: 0
Number of empties: 0


{'empty': [], 'missing': []}

## Finding all SIPSim pipeline incompletes

In [126]:
pipeline_me = find_missing_files('*/*/*/OTU_abs1e9.txt')   

Union length: 200
Number of missing: 0
Number of empties: 1


In [128]:
pipeline_me

{'empty': ['/home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/100/25/6/OTU_abs1e9.txt'],
 'missing': []}

In [134]:
# rerunning 
exe = '/home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/100/25/6/SIPSimRun_SIPSim-pipeline.sh'
exe += ' 2> /home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/100/25/6/SIPSim_pipeline.log'
!$exe 
%pushnote SIPSim pipeline rerun complete

#-- SIPSim pipeline --#
# Adding diffusion
# Adding DBL contamination; abundance-weighted smearing
# Adding isotope incorporation to BD distribution
# Simulating an OTU table
# Simulating PCR
# Subsampling from the OTU table (simulating sequencing of the DNA pool)
# Making a wide-formatted table
# Making metadata (phyloseq: sample_data)


In [135]:
exe = '/home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/100/25/6/SIPSimRun_SIPSim-summary.sh'
exe += ' 2> /home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/100/25/6/SIPSim_summary.log'
!$exe 
%pushnote pipeline summary rerun complete

In [136]:
exe = '/home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/100/25/6/SIPSimRun_HRSIP.sh'
exe += ' 2> /home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/100/25/6/HR-SIP.log'
!$exe 
%pushnote HRSIP rerun complete

In [160]:
pipeline_me = find_missing_files('*/*/*/OTU_abs1e9.txt')  

Union length: 200
Number of missing: 0
Number of empties: 0


## Finding all HR-SIP incompletes

In [165]:
HRSIP_me = find_missing_files('*/*/*/OTU_abs1e9_PCR_sub_filt_DESeq2') 

Union length: 200
Number of missing: 74
Number of empties: 0


In [166]:
HRSIP_me

{'empty': [],
 'missing': ['/home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/25/50/1',
  '/home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/25/5/3',
  '/home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/25/50/3',
  '/home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/25/50/2',
  '/home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/25/50/5',
  '/home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/25/50/4',
  '/home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/25/50/7',
  '/home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/25/50/6',
  '/home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/25/50/9',
  '/home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/25/50/8',
  '/home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/25/25/9',
  '/home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/25/1/10',
  '/

In [ ]:
def run_HRSIP(dirpath):
    cmd = os.path.join(dirpath, 'SIPSimRun_HRSIP.sh')
    log = os.path.join(dirpath, 'HR-SIP.log')    
    !$cmd

for FL in HRSIP_me.values():
    for D in FL:
        print 'Processing: {}'.format(F)
        run_HRSIP(D)

Processing: /home/nick/notebook/SIPSim/dev/bac_genome1147/atomIncorp_taxaIncorp/25/5/9
Warning messages:
1: replacing previous import by ‘scales::alpha’ when loading ‘phyloseq’ 
2: replacing previous import by ‘ggplot2::Position’ when loading ‘DESeq2’ 
Error in read.table(file = file, header = header, sep = sep, quote = quote,  : 
  no lines available in input
Calls: read.delim -> read.table
Execution halted
Warning messages:
1: replacing previous import by ‘scales::alpha’ when loading ‘phyloseq’ 
2: replacing previous import by ‘ggplot2::Position’ when loading ‘DESeq2’ 
Error in readRDS(opts[["<phyloseq>"]]) : error reading from connection
Execution halted
Warning messages:
1: replacing previous import by ‘scales::alpha’ when loading ‘phyloseq’ 
2: replacing previous import by ‘ggplot2::Position’ when loading ‘DESeq2’ 
Error in readRDS(opts[["<phyloseq>"]]) : error reading from connection
Execution halted
^CTraceback (most recent call last):
  File "/home/nick/notebook/SIPSim/SIPSimR"

## Finding all qSIP pipeline incompletes

In [137]:
qSIP_me = find_missing_files('*/*/*/*qSIP_atom.txt')  

Union length: 200
Number of missing: 80
Number of empties: 1


### re-running qSIP

In [ ]:
def run_qSIP(dirpath):
    cmd = os.path.join(dirpath, 'SIPSimRun_qSIP.sh')
    log = os.path.join(dirpath, 'qSIP.log')    
    cmd = '{} 2> {}'.format(cmd, log)
    #!cmd

for FL in qSIP_me.values():
    for D in FL:
        print 'Processing: {}'.format(D)

### re-running cMtx